# Surface concentrations

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import cmocean
import netCDF4 as nc
import modules.mapping_functions as mf
import pandas as pd

%matplotlib inline

#### Parameters:

In [ ]:
# domain dimensions:
imin, imax = 1479, 2179
jmin, jmax = 159, 799
isize = imax - imin
jsize = jmax - jmin

# model results:
year = 2015
results_folder1 = f'/data/brogalla/run_storage/Mn-reference-202204/ANHA12_ref-{year}_20220422/'

#### Load files:

In [ ]:
# ANHA12 grid
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
ANHA12_lon = np.array(mesh.variables['nav_lon'])
ANHA12_lat = np.array(mesh.variables['nav_lat'])
mesh_bathy = np.array(mesh.variables['hdept'][0])
tmask      = np.array(mesh.variables['tmask'])
land_mask  = np.ma.masked_where((tmask[0,:,:,:] > 0.1) , tmask[0,:,:,:]) 

tmask_sub  = np.array(mesh.variables['tmask'])[0,:,imin:imax,jmin:jmax]
e3t        = np.array(mesh.variables['e3t_0'])[0,:,imin:imax,jmin:jmax]
e3t_masked = np.ma.masked_where((tmask_sub[:,:,:] < 0.1), e3t)

In [ ]:
# Load Mn model run results
ref   = nc.Dataset(f'{results_folder1}ANHA12_EXH006_{year}_monthly.nc',  'r')
lons_model  = np.array(ref.variables['nav_lon'])
lats_model  = np.array(ref.variables['nav_lat'])
depth_model = np.array(ref.variables['deptht'])

results  = nc.Dataset(f'{results_folder1}ANHA12_EXH006_{year}_monthly.nc')
dissolmn = np.array(results.variables['dissolmn'][:,0,:,:,:])*10**9 # nM units
oxidismn = np.array(results.variables['oxidismn'][:,0,:,:,:])*10**9 # nM units

In [ ]:
# GEOTRACES 2021 Intermediate data product: https://www.bodc.ac.uk/geotraces/data/idp2021/
folder_data = '/ocean/brogalla/GEOTRACES/data/'
sw          = xr.open_dataset(f'{folder_data}IDP2021_GEOTRACES_IDP2021_Seawater_Discrete_Sample_Data_v1_kYw7mpQ2.nc')
GTR_cruises = sw['metavar1']
GTR_lons    = sw['longitude'].values # degrees east
GTR_lats    = sw['latitude'].values  # degrees north
GTR_pres    = sw['var1'].values      # Pressure from CTD sensor [dbar]
GTR_depth   = sw['var2'].values      # Depth below sea surface calculated from pressure [m]
GTR_temp    = sw['var3'].values      # Temperature from CTD sensor in the ITS-90 convention. [C]
GTR_sal     = sw['var4'].values      # Practical salinity from CTD sensor on the PSS-1978 scale
GTR_dMn     = sw['var12'].values     # Concentration of dissolved Mn in nmol/kg, method may include Mn(II) plus Mn(III)

GTR_pressure   = np.ma.masked_where(GTR_pres < -1e8, GTR_pres)
# practical salinity --> absolute salinity (g/kg)
GTR_lons_tile  = np.tile(GTR_lons, (345,1)).transpose()
GTR_lats_tile  = np.tile(GTR_lats, (345,1)).transpose()
GTR_sal_abs    = gsw.conversions.SA_from_SP(GTR_sal, GTR_pressure, GTR_lons_tile, GTR_lats_tile)
# calculate density: uses in situ temperature, absolute salinity, sea pressure 
GTR_density    = gsw.density.rho_t_exact(GTR_sal_abs, GTR_temp, GTR_pressure) # kg/m3

# convert dMn from nmol/kg --> nmol/L (nmol/kg * kg/m3 * 1 m3 / 1000 L)
GTR_dMn_nM = GTR_dMn*GTR_density*1e-3

GTR_dMn_PML = np.ma.masked_where((GTR_depth > 35), GTR_dMn_nM)
GTR_dMn_PML = np.ma.masked_where(np.isnan(GTR_dMn_nM), GTR_dMn_PML)
GTR_dMn_PML_mean = GTR_dMn_PML.mean(axis=1)

In [ ]:
# Concentration of dMn in the Beaufort Sea in 2009 from Sim, 2018 (doi: 10.14288/1.037422)
IPY = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Nari_Sim_dMn_data.csv')
IPY_names_full = np.array(IPY['Station name'].astype('string'))
IPY_depths     = IPY['Depth [m]'].astype('float').values
IPY_dMn_nmolkg = IPY['DMn [ nmol/kg]'].astype('float').values
IPY_lons_full  = -1*IPY['Longitude'].astype('float').values
IPY_lats_full  = IPY['Latitude'].astype('float').values

# Convert nmol / kg to nmol / L
IPY_density             = IPY['Density'].astype('float').values # kg / L
IPY_depths_cruise_sheet = IPY['Depth [m] cruise sheet'].astype('float').values
IPY_dMn_nmolL = IPY_dMn_nmolkg * IPY_density * 1e-3

IPY_dMn_PML = np.ma.masked_where(IPY_depths > 35, IPY_dMn_nmolL)
IPY_stns = list(dict.fromkeys(IPY_names_full))

IPY_dMn_PML_mean = np.zeros(len(IPY_stns))
for i, stn in enumerate(IPY_stns):
    IPY_dMn_PML_mean[i] = np.ma.mean(IPY_dMn_PML[IPY_names_full==stn])
    
IPY_lons_mean = list(dict.fromkeys(IPY_lons_full))
IPY_lats_mean = list(dict.fromkeys(IPY_lats_full))

#### Functions:

In [ ]:
def plot_layer(plot_variable, vmin=0,vmax=35, savefig=False, obs=True):
    
    fig, ax1, proj1 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/mn-reference.pickle','rb'))

    x, y = proj1(lons_model, lats_model)

    pcolor  = proj1.pcolormesh(x, y, plot_variable, cmap=cmocean.cm.dense, vmin=vmin, vmax=vmax)
    contour = proj1.contourf(np.zeros_like(x), np.zeros_like(y), np.zeros_like(plot_variable), np.arange(vmin,vmax+1,1),\
                             cmap=cmocean.cm.dense, vmin=vmin, vmax=vmax, extend='both')

    
    # Plot observations over top:
    # mask outside of domain
    if obs:
        GTR_lons_ma = np.ma.masked_where((GTR_lats > 85) | (GTR_lons > 290) | ((GTR_lons < 200) & ((GTR_lats > 78))) \
                                         | ((GTR_lons < 203) & (GTR_lats < 74)), GTR_lons)
        GTR_lats_ma = np.ma.masked_where((GTR_lats > 85) | (GTR_lons > 290) | ((GTR_lons < 200) & ((GTR_lats > 78))) \
                                         | ((GTR_lons < 203) & (GTR_lats < 74)), GTR_lats)
        x_GTR, y_GTR = proj1(GTR_lons_ma, GTR_lats_ma)

        # all GEOTRACES: # exclude one point from GIPY11
        proj1.scatter(x_GTR[GTR_cruises != b'GIPY11'], y_GTR[GTR_cruises != b'GIPY11'], c=GTR_dMn_PML_mean[GTR_cruises != b'GIPY11'], \
                      s=50, zorder=3, edgecolor='w', marker='.', \
                      linewidth=0.5, vmin=0, vmax=vmax, cmap=cmocean.cm.dense, label='2015 GEOTRACES GN02, GN03')
        
        # US GEOTRACES:
        proj1.scatter(x_GTR[GTR_cruises == b'GN01'], y_GTR[GTR_cruises == b'GN01'], c=GTR_dMn_PML_mean[GTR_cruises == b'GN01'], \
                      s=13, zorder=3, edgecolor='w', marker='s', \
                      linewidth=0.5, vmin=0, vmax=vmax, cmap=cmocean.cm.dense, label='2015 GEOTRACES GN01')
    
        # IPY: 
        x_IPY, y_IPY = proj1(IPY_lons_mean, IPY_lats_mean)    
        proj1.scatter(x_IPY, y_IPY, c=IPY_dMn_PML_mean, s=13, zorder=3, edgecolor='w', marker='>', \
                      linewidth=0.5, vmin=0, vmax=vmax, cmap=cmocean.cm.dense, label='2009 GIPY14')
    
    leg = ax1.legend(loc=(0.57,0.82), fontsize=5.5, frameon=True)
    leg.get_frame().set_linewidth(0.0)
    leg.legendHandles[0].set_color('#765ec3')
    leg.legendHandles[1].set_color('#765ec3')
    leg.legendHandles[2].set_color('#765ec3')

    cbar_ax = fig.add_axes([0.77, 0.26, 0.04, 0.45])
    cbar    = fig.colorbar(contour, cax=cbar_ax)
    cbar.set_label('Dissolved Mn [nM]', fontsize=6) 
    cbar_ax.tick_params(axis='both', which='major', labelsize=6)
      
    if savefig:
        fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/R5-surface-fields-August.jpg', \
                       bbox_inches='tight', dpi=300)
        fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/R5-surface-fields-August.svg', \
                       bbox_inches='tight', dpi=300, format='svg')
    
    return

In [ ]:
def calculate_depth_weighted(dMn, levels=(0,16)):
    
    dMn_weighted     = dMn[levels[0]:levels[1],:,:]*e3t[levels[0]:levels[1],:,:]
    
    # bottom depth (in case of regions shallower than levels[1]: 
    depth_top        = np.zeros_like(mesh_bathy[imin:imax,jmin:jmax])
    if levels[0] != 0:
        depth_top[mesh_bathy[imin:imax,jmin:jmax] > depth_model[levels[0]-1]] = depth_model[levels[0]-1] # top of layer is not the surface
    depth_bottom     = np.copy(mesh_bathy[imin:imax,jmin:jmax])
    depth_bottom[depth_bottom > depth_model[levels[1]-1]] = depth_model[levels[1]-1]
    delta_depth      =  depth_bottom - depth_top
                 
    # calculate average:
    dMn_weighted_ave = np.sum(dMn_weighted, axis=0) / delta_depth
    
    return dMn_weighted_ave

#### Figures of surface concentrations:

In [ ]:
month=7
plot_variable = calculate_depth_weighted(dissolmn[month,:,:,:], levels=(3,16))
plot_layer(plot_variable,vmin=0,vmax=15, savefig=True)

In [ ]:
month=0
plot_variable = calculate_depth_weighted(dissolmn[month,:,:,:], levels=(0,16))
# plot_variable = dissolmn[month,0,:,:]
plot_layer(plot_variable,vmin=0,vmax=15, savefig=True, obs=False)